In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
import json

In [2]:
# project directory
current_directory = os.getcwd()
folder = current_directory.split("\\")[-1]
if folder != 'galizia_weather':
    os.chdir('..')
    print(os.getcwd()) 
else:
    print(os.getcwd())


c:\projects\galizia_weather


In [3]:
path = "data/raw/Santiago-EOAS_diaria_01-01-2025_to_01-04-2025.csv"
df = pd.read_csv(path, delimiter=";", skiprows=2)
print(df.shape)
df.head()

(273, 5)


,Fecha,Variable,Valor,Código validación,Unidad
0,01/01/2025,Lluvia,0.00,1,L/m2
1,01/01/2025,Humedad relativa media a 1.5m,73.00,1,%
2,01/01/2025,Temperatura media atura a 1.5m,6.43,1,ºC
3,02/01/2025,Lluvia,0.00,1,L/m2
4,02/01/2025,Humedad relativa media a 1.5m,75.00,1,%


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Fecha              273 non-null    object 
 1   Variable           273 non-null    object 
 2   Valor              273 non-null    float64
 3   Código validación  273 non-null    int64  
 4   Unidad             273 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 10.8+ KB


In [26]:
df["Fecha"] = pd.to_datetime(df["Fecha"], dayfirst=True)
df["Valor"] = pd.to_numeric(df["Valor"], errors="coerce")

In [27]:
df_pivot = df.pivot_table(
    index="Fecha",
    columns="Variable",
    values="Valor",
    aggfunc="first"
)

In [28]:
df_pivot

Variable,Humedad relativa media a 1.5m,Lluvia,Temperatura media atura a 1.5m
Fecha,,,
2025-01-01,73.0,0.0,6.43
2025-01-02,75.0,0.0,11.06
2025-01-03,95.0,27.3,12.00
2025-01-04,85.0,12.8,12.71
2025-01-05,92.0,46.8,12.69
...,...,...,...
2025-03-28,78.0,0.0,10.72
2025-03-29,73.0,0.0,11.74
2025-03-30,58.0,0.0,14.04


In [29]:
df_pivot.columns = [col.strip().split(" ")[0].lower() for col in df_pivot.columns]
df_pivot = df_pivot.rename(columns={
    "temperatura": "temperatura",
    "humedad": "humedad",
    "lluvia": "precipitacion"
})
df_pivot = df_pivot.reset_index()
df_pivot = df_pivot.rename(columns={"Fecha":"fecha"})

In [30]:
print(df.info())
df_pivot

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Fecha              273 non-null    datetime64[ns]
 1   Variable           273 non-null    object        
 2   Valor              273 non-null    float64       
 3   Código validación  273 non-null    int64         
 4   Unidad             273 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 10.8+ KB
None


,fecha,humedad,precipitacion,temperatura
0,2025-01-01,73.0,0.0,6.43
1,2025-01-02,75.0,0.0,11.06
2,2025-01-03,95.0,27.3,12.00
3,2025-01-04,85.0,12.8,12.71
4,2025-01-05,92.0,46.8,12.69
...,...,...,...,...
86,2025-03-28,78.0,0.0,10.72
87,2025-03-29,73.0,0.0,11.74
88,2025-03-30,58.0,0.0,14.04
89,2025-03-31,53.0,0.0,17.21


## GRÁFICAS EN PLOTLY

In [37]:
# 2. Visualizaciones

# Temperatura diaria
fig_temp = px.line(df_pivot, x="fecha", y="temperatura", title="Temperatura media diaria en Santiago")
fig_temp.show()

# Lluvia diaria
fig_rain = px.bar(df_pivot, x="fecha", y="precipitacion", title="Lluvia diaria en Santiago")
fig_rain.show()

# Temperatura + lluvia juntas
fig_combo = go.Figure()
fig_combo.add_trace(go.Bar(x=df_pivot["fecha"], y=df_pivot["precipitacion"], name="Lluvia (mm)", yaxis="y", marker_color="rgba(0,0,255,0.4)"))
fig_combo.add_trace(go.Scatter(x=df_pivot["fecha"], y=df_pivot["temperatura"], name="Temperatura (°C)", yaxis="y2", mode='lines', line=dict(color="red")))

fig_combo.update_layout(
    title="Lluvia y Temperatura diarias en Santiago",
    yaxis=dict(title="Lluvia (mm)"),
    yaxis2=dict(title="Temperatura (°C)", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99),
    height=500
)
fig_combo.show()

# Lluvia mensual
df_pivot["mes"] = df_pivot["fecha"].dt.to_period("M").astype(str)
rain_monthly = df_pivot.groupby("mes")["precipitacion"].sum().reset_index()
fig_rain_month = px.bar(rain_monthly, x="mes", y="precipitacion", title="Lluvia acumulada por mes")
fig_rain_month.update_xaxes(dtick="M1", tickformat="%b %Y")
fig_rain_month.show()

# Temperatura mensual
temp_monthly = df_pivot.groupby("mes")["temperatura"].mean().reset_index()
fig_temp_month = px.line(temp_monthly, x="mes", y="temperatura", title="Temperatura media mensual")
fig_temp_month.update_xaxes(dtick="M1", tickformat="%b %Y")
fig_temp_month.show()

# Histograma
fig_hist = px.histogram(df_pivot, x="temperatura", nbins=30, title="Distribución de temperaturas")
fig_hist.update_xaxes(dtick="M1", tickformat="%b %Y")
fig_hist.show()

# Boxplot por mes
fig_box = px.box(df_pivot, x="mes", y="temperatura", title="Boxplot mensual de temperatura")
fig_box.update_xaxes(dtick="M1", tickformat="%b %Y")
fig_box.show()

In [34]:
df_pivot.to_csv('data/processed/weather_santiago.csv')

In [43]:
df_pivot

,Fecha,humedad_media,lluvia,temperatura_media,Mes
0,2025-01-01,73.0,0.0,6.43,2025-01
1,2025-01-02,75.0,0.0,11.06,2025-01
2,2025-01-03,95.0,27.3,12.00,2025-01
3,2025-01-04,85.0,12.8,12.71,2025-01
4,2025-01-05,92.0,46.8,12.69,2025-01
...,...,...,...,...,...
86,2025-03-28,78.0,0.0,10.72,2025-03
87,2025-03-29,73.0,0.0,11.74,2025-03
88,2025-03-30,58.0,0.0,14.04,2025-03
89,2025-03-31,53.0,0.0,17.21,2025-03


In [47]:
# Definir si llovió o no
df_pivot["llovio"] = df_pivot["lluvia"] > 0

# Contar días con y sin lluvia
dias_lluvia = df_pivot["llovio"].value_counts().rename({True: "Día con lluvia", False: "Día sin lluvia"}).reset_index()
dias_lluvia.columns = ["Tipo de día", "Cantidad"]

# Pie chart
fig_pie = px.pie(
    dias_lluvia,
    names="Tipo de día",
    values="Cantidad",
    title="Porcentaje de días con lluvia vs sin lluvia",
    hole=0.4  # para hacerlo tipo donut
)
fig_pie.update_traces(textinfo="percent+label")
fig_pie.show()


In [48]:
dias_con_lluvia = (df_pivot["lluvia"] > 0).sum()
total_dias = len(df_pivot)

print(f"Días con lluvia: {dias_con_lluvia} de {total_dias} ({dias_con_lluvia / total_dias:.1%})")


Días con lluvia: 48 de 91 (52.7%)


In [ ]:
# Contar días con lluvia por mes
dias_lluviosos_por_mes = df_pivot[df_pivot["lluvia"] > 0].groupby("Mes").size().reset_index(name="Días con lluvia")

# Ordenar para ver el mes con más días de lluvia
dias_lluviosos_por_mes = dias_lluviosos_por_mes.sort_values(by="Días con lluvia", ascending=False)

# Mostrar tabla
print(dias_lluviosos_por_mes)

fig_dias_lluvia = px.bar(
    dias_lluviosos_por_mes,
    x="Mes",
    y="Días con lluvia",
    title="Cantidad de días con lluvia por mes"
)
fig_dias_lluvia.update_xaxes(type="category")
fig_dias_lluvia.show()

       Mes  Días con lluvia
0  2025-01               20
1  2025-02               16
2  2025-03               12


In [50]:
# Suma de lluvia por mes
lluvia_total_por_mes = df_pivot.groupby("Mes")["lluvia"].sum().reset_index(name="Lluvia total (mm)")

# Ordenar
lluvia_total_por_mes = lluvia_total_por_mes.sort_values(by="Lluvia total (mm)", ascending=False)

# Mostrar tabla
print(lluvia_total_por_mes)

# Visualizar con barras
fig_lluvia_total = px.bar(
    lluvia_total_por_mes,
    x="Mes",
    y="Lluvia total (mm)",
    title="Cantidad total de lluvia por mes"
)
fig_lluvia_total.update_xaxes(type="category")
fig_lluvia_total.show()


       Mes  Lluvia total (mm)
0  2025-01              514.0
1  2025-02              139.8
2  2025-03              113.0
3  2025-04                0.0


In [39]:
df_pivot[df_pivot["precipitacion"] != 0]

,fecha,humedad,precipitacion,temperatura,mes
2,2025-01-03,95.0,27.3,12.00,2025-01
3,2025-01-04,85.0,12.8,12.71,2025-01
4,2025-01-05,92.0,46.8,12.69,2025-01
5,2025-01-06,91.0,5.7,7.97,2025-01
6,2025-01-07,95.0,35.4,11.40,2025-01
7,2025-01-08,89.0,62.1,13.14,2025-01
8,2025-01-09,91.0,4.8,13.38,2025-01
9,2025-01-10,97.0,21.9,13.96,2025-01
10,2025-01-11,95.0,1.9,13.76,2025-01
18,2025-01-19,87.0,14.6,7.89,2025-01


In [5]:
# project directory
current_directory = os.getcwd()
folder = current_directory.split("\\")[-1]
if folder != 'galizia_weather':
    os.chdir('..')
    print(os.getcwd()) 
else:
    print(os.getcwd())

folder = 'data/processed'
dataframes = []
for archivo in os.listdir(folder):
        if archivo.endswith('.csv'):
            path_folder = os.path.join(folder, archivo)
            df = pd.read_csv(path_folder, index_col=0)
            # Agregar columna con el nombre del archivo (sin extensión)
            nombre_ciudad = os.path.splitext(archivo)[0]
            df['ciudad'] = nombre_ciudad
        
            dataframes.append(df)
df_final = pd.concat(dataframes)

# Mostrar los primeros registros
print(df_final.head())

c:\projects\galizia_weather
        fecha  humedad  precipitacion  temperatura  ciudad
0  2023-01-01     98.0           22.6        12.01  coruna
1  2023-01-02     90.0            1.1        10.98  coruna
2  2023-01-03     86.0            0.0        12.01  coruna
3  2023-01-04     91.0            0.0        14.55  coruna
4  2023-01-05     95.0            0.0        12.99  coruna


In [ ]:
MESES_ORDENADOS = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

def variables_de_tiempo(df):
    df["fecha"] = pd.to_datetime(df["fecha"])
    # Asegúrate de que las columnas necesarias existan
    df["year"] = df["fecha"].dt.year
    df["mes_num"] = df["fecha"].dt.month
    df["mes_nombre"] = df["mes_num"].map(MESES_ORDENADOS)
    df["llovio"] = df["precipitacion"] > 0
    df['mes_anyo'] = df['fecha'].dt.to_period('M').dt.to_timestamp()
    return df

df = variables_de_tiempo(df_final)
df.head()

,fecha,humedad,precipitacion,temperatura,ciudad,year,mes_num,mes_nombre,llovio,mes_anyo
0,2023-01-01,98.0,22.6,12.01,coruna,2023,1,Enero,True,2023-01
1,2023-01-02,90.0,1.1,10.98,coruna,2023,1,Enero,True,2023-01
2,2023-01-03,86.0,0.0,12.01,coruna,2023,1,Enero,False,2023-01
3,2023-01-04,91.0,0.0,14.55,coruna,2023,1,Enero,False,2023-01
4,2023-01-05,95.0,0.0,12.99,coruna,2023,1,Enero,False,2023-01


In [11]:
localizacion = 'Galicia'
if 'ciudad' in df.columns:
        df_mensual = df.groupby(['mes_anyo', 'ciudad'], as_index=False)['precipitacion'].sum()
        fig = px.bar(df_mensual, x="mes_anyo", y="precipitacion", color="ciudad",
                    title=f"         Precipitación mensual en {localizacion}")
else:
    df_mensual = df.groupby('mes_anyo', as_index=False)['precipitacion'].sum()
    fig = px.bar(df_mensual, x="mes_anyo", y="precipitacion",
                title=f"         Precipitación mensual en {localizacion}")

# Estilo del gráfico
fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='white'),
    title_font=dict(color='white'),
    legend=dict(font=dict(color='white')),
    xaxis=dict(title='Mes', color='white'),
    yaxis=dict(title='Precipitación mensual (L/m2)', color='white', gridcolor='rgba(255, 255, 255, 0.4)'),
    autosize=True,
    margin=dict(l=20, r=20, t=40, b=40)
)

fig.show()

TypeError: Object of type Period is not JSON serializable

In [12]:
def clasificar_temperatura(temp):
    if temp < 10:
        return 'Frío'
    elif temp < 20:
        return 'Templado'
    else:
        return 'Calor'

df_final['categoria_temp'] = df_final['temperatura'].apply(clasificar_temperatura)
categorias_ordenadas = ["Frío", "Templado", "Calor"]
df_final["categoria_temp"] = pd.Categorical(df_final["categoria_temp"], categories=categorias_ordenadas, ordered=True)
df_temp_cat = df_final.groupby("categoria_temp").size().reset_index(name='count')

fig_temp_cat = px.bar(
    df_temp_cat,
    x='categoria_temp',
    y='count',
    labels={'index': 'Categoría de Temperatura', 'categoria_temp': 'Frecuencia'},
    title='Distribución de Categorías de Temperatura'
)

fig_temp_cat.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='white'),
    title_font=dict(color='white')
)

fig_temp_cat.show()
df_temp_cat

C:\Users\alfre\AppData\Local\Temp\ipykernel_19724\2782417356.py:12: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,categoria_temp,count
0,Frío,870
1,Templado,3304
2,Calor,752


In [5]:
df_final['categoria_temp'].value_counts().reset_index()

,categoria_temp,count
0,Templado,3304
1,Frío,870
2,Calor,752


In [5]:
localizacion = "Galicia"
multiple_cities = df_final['ciudad'].nunique() > 1

if multiple_cities:
    fig = px.line(df_final, x="fecha", y="temperatura", color="ciudad",
                    title=f"Temperatura diaria en {localizacion}")
else:
    fig = px.line(df_final, x="fecha", y="temperatura",
                    title=f"Temperatura diaria en {localizacion}")

fig.update_layout(
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    font=dict(color='white'),
    title_font=dict(color='white'),
    legend=dict(font=dict(color='white')),
    xaxis=dict(title='Fecha', color='white'),
    yaxis=dict(title='Temperatura (°C)', color='white', gridcolor='rgba(255, 255, 255, 0.4)'),
    autosize=True,
    margin=dict(l=20, r=20, t=40, b=40)
)

fig.show()

In [7]:
def plot_temp_line(df_filtrado, localizacion):
    # Detectar si hay más de una ciudad
    multiple_cities = df_filtrado['ciudad'].nunique() > 1

    if multiple_cities:
        fig = px.line(df_filtrado, x="fecha", y="temperatura", color="ciudad",
                      title=f"Temperatura diaria en {localizacion}")
    else:
        fig = px.line(df_filtrado, x="fecha", y="temperatura",
                      title=f"Temperatura diaria en {localizacion}")

    fig.update_layout(
        plot_bgcolor='rgba(0, 0, 0, 0)',
        paper_bgcolor='rgba(0, 0, 0, 0)',
        font=dict(color='white'),
        title_font=dict(color='white'),
        legend=dict(font=dict(color='white')),
        xaxis=dict(title='Fecha', color='white'),
        yaxis=dict(title='Temperatura (°C)', color='white', gridcolor='rgba(255, 255, 255, 0.4)'),
        autosize=True,
        margin=dict(l=20, r=20, t=40, b=40)
    )
    
    return fig

fig_temp = (df_final, localizacion)
fig_temp.show()

AttributeError: 'tuple' object has no attribute 'show'

In [64]:
MESES_ORDENADOS = {
    1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril",
    5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto",
    9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

df_final["fecha"] = pd.to_datetime(df_final["fecha"])
df_final["mes_num"] = df_final["fecha"].dt.month
meses_disponibles = sorted(df_final["mes_num"].unique())
[MESES_ORDENADOS[m] for m in meses_disponibles]

['Enero',
 'Febrero',
 'Marzo',
 'Abril',
 'Mayo',
 'Junio',
 'Julio',
 'Agosto',
 'Septiembre',
 'Octubre',
 'Noviembre',
 'Diciembre']

In [76]:
df_grouped = df.groupby('fecha').agg({
    'temperatura': 'mean',
    'precipitacion': 'sum',
    'humedad': 'mean'
}).reset_index()
df_grouped

,fecha,temperatura,precipitacion,humedad
0,2023-01-01,13.06,54.2,90.0
1,2023-01-02,10.89,3.0,81.0
2,2023-01-03,11.28,0.0,79.0
3,2023-01-04,12.15,0.0,83.0
4,2023-01-05,11.40,0.0,84.0
...,...,...,...,...
816,2025-03-27,13.31,0.0,80.0
817,2025-03-28,14.30,0.0,71.0
818,2025-03-29,16.12,0.0,54.0
819,2025-03-30,16.74,0.0,56.0


In [78]:
df_grouped['llovio'] = df_grouped['precipitacion'] > 0
conteo_con = df_grouped["llovio"].sum()
conteo_sin = len(df_grouped) - conteo_con
conteo = [conteo_con, conteo_sin]
conteo = [conteo_con, conteo_sin]
etiquetas = ['Días con lluvia 🌧️', 'Días sin lluvia ☀️']

df_conteo = pd.DataFrame({
    'Día': etiquetas,
    'Cantidad': conteo
})


In [53]:
[int(i) for i in meses_disponibles]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [50]:
df_final["year"] = df_final["fecha"].dt.year
años = sorted(df_final["year"].unique(), reverse=True)
años

[np.int32(2025), np.int32(2024), np.int32(2023)]

In [33]:
df_final[['precipitacion', 'humedad', 'temperatura', 'fecha']].groupby('fecha')['precipitacion'].sum().max()

np.float64(322.3)

In [39]:
df_final[['precipitacion', 'humedad', 'temperatura', 'fecha']].groupby('fecha')['precipitacion'].sum().max()

np.float64(322.3)

In [23]:
df_final['precipitacion'].min()

np.float64(0.0)

In [38]:
df_final["llovio"] = df_final["precipitacion"] > 0
conteo = df_final["llovio"].value_counts().rename({True: "Días con lluvia 🌧️", False: "Días sin lluvia ☀️"}).reset_index()
conteo.columns = ["Tipo de día", "Cantidad"]
conteo

,Tipo de día,Cantidad
0,Días sin lluvia ☀️,2867
1,Días con lluvia 🌧️,2059


In [41]:
print(conteo.iloc[0][0])

Días sin lluvia ☀️


C:\Users\alfre\AppData\Local\Temp\ipykernel_2156\346982798.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(conteo.iloc[0][0])


In [42]:
conteo.iloc[0][0] == 'Días sin lluvia ☀️'

C:\Users\alfre\AppData\Local\Temp\ipykernel_2156\26337446.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  conteo.iloc[0][0] == 'Días sin lluvia ☀️'


True

In [3]:
'Coruna'.lower().replace('ñ', 'n')

'coruna'

In [12]:
current_directory = os.getcwd()
folder = current_directory.split("\\")[-1]
if folder != 'galizia_weather':
    os.chdir('..')
    print(os.getcwd()) 
else:
    print(os.getcwd())

path = 'data/raw/city_coordinates.json'

with open(path, "r") as f:
    data = json.load(f)

# Mostrar el contenido del JSON
print(json.dumps(data, indent=4)) 

c:\projects\galizia_weather
[
    [
        "Santiago de Compostela",
        "71938",
        [
            -8.5434,
            42.88187
        ]
    ],
    [
        "Coruna",
        "71933",
        [
            -8.3949,
            43.34986
        ]
    ],
    [
        "Lugo",
        "71940",
        [
            -7.5562,
            43.01375
        ]
    ],
    [
        "Pontevedra",
        "71954",
        [
            -8.62686,
            42.42331
        ]
    ],
    [
        "Ourense",
        "71953",
        [
            -7.86336,
            42.33942
        ]
    ],
    [
        "Vigo",
        "71956",
        [
            -8.74281,
            42.21245
        ]
    ]
]


In [14]:
data['Santiago de Compostela']

TypeError: list indices must be integers or slices, not str

In [24]:
dict_ = {'clave': 'valor'}

print(dict_.keys())

dict_keys(['clave'])
